**Load the libraries**

In [5]:
from pyprojroot import here
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import (set_global_service_context,
                         ServiceContext,
                         SimpleDirectoryReader,
                         VectorStoreIndex,
                         StorageContext,
                         load_index_from_storage)
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

**Load the GPT model and the embedding model from AzureOpenAI**

In [6]:
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    engine="gpt-35-turbo-16k",
    deployment_name=os.getenv("gpt_deployment_name"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name=os.getenv("embed_deployment_name"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
# NOTE: Uncomment if you want to use an open source embedding model
# embed_model = "local:BAAI/bge-small-en-v1.5"

**Set the serivce context**

In [8]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)


**Load the documents**

In [ ]:
documents = SimpleDirectoryReader(
    input_files=[here(f"data/docs/{d}") for d in os.listdir(here("data/docs"))]
).load_data()

**Process documents and prepare the index:**

Processing for `normal RAG`: Each chunk is one page of the documents

In [ ]:
save_dir=here("data/indexes/pagewise_index")

In [ ]:
index = VectorStoreIndex.from_documents(documents)

**Save the index if you wish**

In [ ]:
index.storage_context.persist(save_dir)

**Load the index**

In [ ]:
# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=save_dir)

# Load index from the storage context
index = load_index_from_storage(storage_context)

**Build the query engine**

In [ ]:
query_engine = index.as_query_engine()

**Test with a query**

In [10]:
answer = query_engine.query("What is most interesting about Fred?")

print(answer.get_formatted_sources())
print("answer:", answer)

> Source (Doc id: f1d6fe33-3ef2-4c65-98b7-6540e490d974): Fred the red fish 
Once upon a time, in the vast, shimmering ocean, there lived a small red fish ...

> Source (Doc id: 6b2e3d7f-076b-4c79-886a-d6221ce090ea): Amarok the lone wolf 
In the shadowed embrace of the Alaskan mountains, where the snow blankets t...
query was: What is most interesting about Fred?
answer was: Fred's most interesting quality is his desire for adventure and his eagerness to explore every nook and cranny of the reef.
